In [9]:
import scipy.io
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import math
import soundfile as sf
import os
from pydub import AudioSegment
from tqdm import tqdm
import random
random.seed(4)


In [10]:
# prendi il nome di tutti i files
audio_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
audio_names = os.listdir(audio_path)
audio_names[:5]

['20190601_000000.WAV',
 '20190601_010000.WAV',
 '20190601_020000.WAV',
 '20190601_030000.WAV',
 '20190601_040000.WAV']

In [11]:
bird_tags = scipy.io.loadmat('Bird_tags_Train.mat')["Bird_tags"]
category_info = {}
audio_info = {}
for elem in bird_tags:
    tag = elem[0][0][0][0][0]
    file_name = elem[0][0][0][1][0]
    bbox = elem[0][0][0][3][:4]
    start_time = math.floor(min(bbox[:, 0]))
    end_time = math.floor(max(bbox[:, 0]))
    duration = end_time - start_time
    if tag not in category_info:
        category_info[tag] = []
    if file_name not in audio_info:
        audio_info[file_name] = []
    category_info[tag].append({"file_name": file_name, "bbox": bbox.tolist(), "start_time": start_time, "duration": duration})
    audio_info[file_name].append({"tag": tag, "start_time": start_time, "duration": duration})


In [12]:
des_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments"
os.makedirs("D:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments\\train", exist_ok=True)
os.makedirs("D:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments\\test", exist_ok=True)

In [13]:
target_dir = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
extracted_info = {}
for category in category_info:
    files_info = category_info[category]
    extracted_info[category] = []
    for file in files_info:
        duration = file["duration"]
        if file["file_name"] not in audio_names or duration < 3:
            continue
        segments_start_time = [ file["start_time"] + i*3 for i in range(duration // 3)]
        full_path = os.path.join(target_dir, file["file_name"])
        for segment in segments_start_time:
            extracted_info[category].append({"file_name": full_path, "start_time": segment})
    random.shuffle(extracted_info[category])

In [14]:
len(extracted_info["Wind"])

126

In [15]:
# impiega 7 minuti
des_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\segments"
categories = list(extracted_info.keys())
# for i in range(len(categories)):
#     category = categories[i]
#     os.makedirs(os.path.join(des_path, "train", category), exist_ok=True)
#     os.makedirs(os.path.join(des_path, "test", category), exist_ok=True)
#     all_category_audio = extracted_info[category]
#     total_length = len(all_category_audio)
#     threshold = 0.7
#     train_length = int(total_length * 0.7)
#     test_length = total_length - train_length
#     print(f"Loading {category} category... {i}/{len(categories)}")
#     for i in tqdm(range(total_length)):
#         full_path = all_category_audio[i]["file_name"]
#         audio = AudioSegment.from_file(full_path, format="wav")
#         start_time = int(all_category_audio[i]["start_time"])
#         segment = audio[start_time:start_time + 3000]
#         split = "train" if i < train_length else "test"
#         file_name = full_path.split("\\")[-1]
#         file_name = file_name[:-4]
#         export_path = os.path.join(des_path, split, category, file_name+f"_{i}"+".wav")
#         segment.export(export_path, format="wav")


Loading Wind category... 0/56


100%|██████████| 126/126 [00:12<00:00, 10.28it/s]


Loading Regulus_ignicapilla category... 1/56


100%|██████████| 296/296 [00:24<00:00, 12.23it/s]


Loading Sylvia_atricapilla category... 2/56


...

0it [00:00, ?it/s]


Loading Poecile_palustris category... 54/56


0it [00:00, ?it/s]


Loading unknown category... 55/56


0it [00:00, ?it/s]
